In [19]:
import requests
import json

# API base URL
BASE_URL = "http://57.159.27.80:8001"

# Client-side mapping for display (questionnaire to human-readable disorder name)
CLIENT_DISORDER_MAPPING = {
    "AQ-10": "Autism Spectrum Disorder",
    "ASRS": "ADHD",
    "GAD-7": "Anxiety",
    "PHQ-9": "Depression",
    "Y-BOCS": "OCD",
    "PCL-5": "PTSD",
    "AUDIT": "Alcohol Use Disorder",
    "DAST": "Drug Use Disorder",
    "Mood Disorder Questionnaire": "Bipolar Disorder",
    "MSI-BPD": "BPD",
    "EAT-26": "Eating Disorders"
}

# Server-side DISORDER_MAPPING (mirroring FastAPI server for payload compatibility)
SERVER_DISORDER_MAPPING = {
    "autismspectrumdisorderasd": "AQ-10",
    "attentiondeficithyperactivitydisorderadhd": "ASRS",
    "anxiety": "GAD-7",
    "depression": "PHQ-9",
    "obsessivecompulsivedisorderocd": "Y-BOCS",
    "posttraumaticstressdisorderptsd": "PCL-5",
    "alcoholusedisorder": "AUDIT",
    "drugusedisorder": "DAST",
    "bipolar": "Mood Disorder Questionnaire",
    "borderlinepersonalitydisorderbpd": "MSI-BPD",
    "eatingdisorder": "EAT-26"
}

def get_questionnaire(disorder_name: str):
    """
    Fetch the questionnaire for a given disorder from the API.
    
    Args:
        disorder_name (str): Name of the disorder (e.g., 'attentiondeficithyperactivitydisorderadhd', 'anxiety')
    
    Returns:
        dict: Questionnaire data if successful, None otherwise
    """
    url = f"{BASE_URL}/get_questionnaire/{disorder_name}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an exception for 4xx/5xx errors
        questionnaire_data = response.json()
        # Fix potential typo in 'instructions' key
        if 'instrcutions' in questionnaire_data and 'instructions' not in questionnaire_data:
            questionnaire_data['instructions'] = questionnaire_data.pop('instrcutions')
        print(f"Questionnaire for {disorder_name} fetched successfully:")
        print(json.dumps(questionnaire_data, indent=2))
        return questionnaire_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching questionnaire for {disorder_name}: {e}")
        return None

def evaluate_responses(payload: dict):
    """
    Evaluate the provided responses for a questionnaire using the /evaluate endpoint.
    
    Args:
        payload (dict): JSON payload with 'questionnaire' (str) and 'responses' (list of dicts)
                       Example: {"questionnaire": "ASRS", "responses": [{"question_id": 1, "option_index": 1}, ...]}
    
    Returns:
        dict: Evaluation result if successful, None otherwise
    """
    url = f"{BASE_URL}/evaluate"
    try:
        # Validate payload structure
        if not isinstance(payload, dict) or "questionnaire" not in payload or "responses" not in payload:
            raise ValueError("Payload must contain 'questionnaire' and 'responses' keys")
        if not isinstance(payload["responses"], list):
            raise ValueError("'responses' must be a list of dictionaries")
        
        response = requests.post(url, json=payload)
        response.raise_for_status()
        evaluation_result = response.json()
        print("\nEvaluation Result:")
        print(json.dumps(evaluation_result, indent=2))
        return evaluation_result
    except requests.exceptions.RequestException as e:
        print(f"Error evaluating responses: {e}")
        if response.status_code == 404:
            print("Possible issue: Questionnaire name may not match expected value in API's DISORDER_MAPPING.")
            print(f"Expected questionnaire names: {list(SERVER_DISORDER_MAPPING.values())}")
        return None
    except ValueError as e:
        print(f"Error evaluating responses: {e}")
        return None

def main(disorder_name: str, payload: dict = None):
    """
    Main function to fetch questionnaire and evaluate responses (if payload provided).
    
    Args:
        disorder_name (str): Name of the disorder to fetch questionnaire
        payload (dict, optional): Payload with responses to evaluate
    """
    # Step 1: Validate disorder name
    if disorder_name not in SERVER_DISORDER_MAPPING:
        print(f"Invalid disorder name: {disorder_name}. Valid options: {list(SERVER_DISORDER_MAPPING.keys())}")
        return
    
    # Step 2: Fetch the questionnaire
    questionnaire_data = get_questionnaire(disorder_name)
    
    if not questionnaire_data:
        print("Cannot proceed due to questionnaire fetch failure.")
        return
    
    # Step 3: Evaluate responses if payload is provided
    if payload:
        # Set the correct questionnaire name from SERVER_DISORDER_MAPPING
        expected_questionnaire = SERVER_DISORDER_MAPPING[disorder_name]
        if payload.get("questionnaire") != expected_questionnaire:
            print(f"Warning: Payload questionnaire '{payload.get('questionnaire')}' does not match expected '{expected_questionnaire}'. Correcting...")
            payload["questionnaire"] = expected_questionnaire
        
        # Validate response question IDs
        expected_question_ids = {q["id"] for q in questionnaire_data.get("questions", [])}
        provided_question_ids = {r["question_id"] for r in payload.get("responses", [])}
        if not provided_question_ids.issubset(expected_question_ids):
            print(f"Error: Invalid question IDs in responses: {provided_question_ids - expected_question_ids}")
            return
        
        # Validate option_index values
        for response in payload.get("responses", []):
            question_id = response["question_id"]
            option_index = response["option_index"]
            question = next((q for q in questionnaire_data["questions"] if q["id"] == question_id), None)
            if question is None:
                print(f"Error: Question ID {question_id} not found in questionnaire")
                return
            if not (0 <= option_index < len(question["options"])):
                print(f"Error: Invalid option_index {option_index} for question ID {question_id}. Valid range: 0–{len(question['options'])-1}")
                return
        
        evaluation_result = evaluate_responses(payload)
        if evaluation_result:
            # Save questionnaire, responses, and evaluation to a file
            output_file = f"{disorder_name}_results.json"
            with open(output_file, "w") as f:
                json.dump({
                    "disorder": CLIENT_DISORDER_MAPPING.get(expected_questionnaire, disorder_name),  # Use human-readable name
                    "questionnaire": expected_questionnaire,
                    "questionnaire_data": questionnaire_data,
                    "responses": payload.get("responses"),
                    "evaluation": evaluation_result
                }, f, indent=2)
            print(f"\nResults saved to {output_file}")

if __name__ == "__main__":
    # Correct disorder name for ADHD
    disorder_name = "attentiondeficithyperactivitydisorderadhd"  # Maps to ASRS
    
    # Sample payload for ASRS (18 questions, options: "Never"=0, "Rarely"=1, "Sometimes"=2, "Often"=3, "Very Often"=4)
    example_payload = {
        "questionnaire": "ASRS",
        "responses": [
            {"question_id": 1, "option_index": 2},  # Sometimes
            {"question_id": 2, "option_index": 3},  # Often
            {"question_id": 3, "option_index": 1},  # Rarely
            {"question_id": 4, "option_index": 2},  # Sometimes
            {"question_id": 5, "option_index": 3},  # Often
            {"question_id": 6, "option_index": 2},  # Sometimes
            {"question_id": 7, "option_index": 1},  # Rarely
            {"question_id": 8, "option_index": 2},  # Sometimes
            {"question_id": 9, "option_index": 2},  # Sometimes
            {"question_id": 10, "option_index": 1}, # Rarely
            {"question_id": 11, "option_index": 2}, # Sometimes
            {"question_id": 12, "option_index": 1}, # Rarely
            {"question_id": 13, "option_index": 3}, # Often
            {"question_id": 14, "option_index": 2}, # Sometimes
            {"question_id": 15, "option_index": 1}, # Rarely
            {"question_id": 16, "option_index": 2}, # Sometimes
            {"question_id": 17, "option_index": 1}, # Rarely
            {"question_id": 18, "option_index": 2}  # Sometimes
        ]
    }
    
    # Run the main function
    main(disorder_name, example_payload)

Questionnaire for attentiondeficithyperactivitydisorderadhd fetched successfully:
{
  "name": "Adult ADHD Self-Report Scale (ASRS)",
  "testname": "ASRS",
  "instructions": "Please read each statement and select the option that best reflects your experiences. Think about how you have felt and behaved over the past 6 months to make your selection.",
  "questions": [
    {
      "id": 1,
      "text": "How often do you have trouble wrapping up the final details of a project, once the challenging parts have been done?",
      "options": [
        "Never",
        "Rarely",
        "Sometimes",
        "Often",
        "Very Often"
      ]
    },
    {
      "id": 2,
      "text": "How often do you have difficulty getting things in order when you have to do a task that requires organization?",
      "options": [
        "Never",
        "Rarely",
        "Sometimes",
        "Often",
        "Very Often"
      ]
    },
    {
      "id": 3,
      "text": "How often do you have problems reme